<a href="https://colab.research.google.com/github/elsedore/daily-analysis/blob/main/dooat_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
import pandas as pd

url = 'https://raw.githubusercontent.com/elsedore/daily-analysis/main/partial_export.csv'

# Reading csv file
df = pd.read_csv(url, sep=';', encoding='utf-8')

# Displaying some information on dataset
print(df.head())
print(df.info())
print(df.describe())

#replacing missing value
df.fillna(0, inplace=True)

# Group by status
df_status = df.groupby('Status')
print(df_status)
print(df_status.head())

   Status           Identifiant  Num�ro de version                  Nom  \
0  Active  05470_EDF_H_00106295                  1      GRAND MAISON 10   
1  Active  05470_EDF_T_00170946                  2             PALUEL 3   
2  Active  05470_EDF_T_00170941                  2  MARTIGUES PONTEAU 5   
3  Active  05470_EDF_T_00170902                  2          CORDEMAIS 4   
4  Active  05470_EDF_T_00171008                  1          TRICASTIN 4   

                                             Fili�re        Date de d�but  \
0  Station de transfert d'�nergie par pompage hyd...  2024-09-02 11:15:00   
1                                          Nucl�aire  2024-09-03 00:00:00   
2                                        Gaz fossile  2024-09-03 00:00:00   
3                                    Houille fossile  2024-09-03 00:00:00   
4                                          Nucl�aire  2024-09-03 00:00:00   

           Date de fin        Type                         Cause  \
0  2024-09-02 17:1